In [ ]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import MinMaxScaler

# Set up visualization style
plt.style.use('seaborn-darkgrid')
sns.set_palette('muted')

# Define the stock symbols and the date range
tickers = ['TSLA', 'BND', 'SPY']
start_date = '2015-01-01'
end_date = '2024-10-31'

# Fetching and Preprocessing
def fetch_data(ticker):
    data = yf.download(ticker, start=start_date, end=end_date)
    data['Ticker'] = ticker
    data.fillna(method='ffill', inplace=True)
    return data

# Scaling function for columns if needed
def scale_data(data, columns):
    scaler = MinMaxScaler()
    data[columns] = scaler.fit_transform(data[columns])
    return data, scaler

# EDA and Visualization Functions
def plot_closing_prices(data, ticker):
    plt.figure(figsize=(14, 7))
    plt.plot(data.index, data['Close'], label=ticker)
    plt.title(f"{ticker} Closing Prices Over Time")
    plt.xlabel("Date")
    plt.ylabel("Scaled Close Price")
    plt.legend()
    plt.show()

def plot_daily_returns(data, ticker):
    data['Daily Return'] = data['Adj Close'].pct_change()
    plt.figure(figsize=(14, 7))
    plt.plot(data.index, data['Daily Return'], label=f'{ticker} Daily Return')
    plt.title(f"{ticker} Daily Percentage Change (Volatility)")
    plt.xlabel("Date")
    plt.ylabel("Daily Percentage Change")
    plt.legend()
    plt.show()

def rolling_statistics(data, ticker):
    data['Rolling Mean'] = data['Adj Close'].rolling(window=30).mean()
    data['Rolling Std'] = data['Adj Close'].rolling(window=30).std()
    plt.figure(figsize=(14, 7))
    plt.plot(data.index, data['Rolling Mean'], label=f'{ticker} 30-Day Rolling Mean')
    plt.plot(data.index, data['Rolling Std'], label=f'{ticker} 30-Day Rolling Std')
    plt.title(f"{ticker} 30-Day Rolling Mean and Standard Deviation")
    plt.xlabel("Date")
    plt.ylabel("Scaled Value")
    plt.legend()
    plt.show()

def seasonality_trend_analysis(data, ticker):
    decomposition = seasonal_decompose(data['Adj Close'], model='multiplicative', period=252)
    decomposition.plot()
    plt.suptitle(f"Seasonal Decomposition of {ticker} Adj Close Price", fontsize=16)
    plt.show()

def risk_analysis(data, ticker):
    data['Daily Return'] = data['Adj Close'].pct_change()
    var = data['Daily Return'].quantile(0.05)
    sharpe_ratio = data['Daily Return'].mean() / data['Daily Return'].std()
    print(f"{ticker} Value at Risk (VaR) at 5%: {var}")
    print(f"{ticker} Sharpe Ratio: {sharpe_ratio}")

# Run EDA on each asset separately
for ticker in tickers:
    data = fetch_data(ticker)
    print(f"\nAnalysis for {ticker}:")

    # Scale Data (optional)
    data, scaler = scale_data(data, ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
    
    # Plot closing prices
    plot_closing_prices(data, ticker)

    # Plot daily returns
    plot_daily_returns(data, ticker)

    # Rolling mean and std deviation
    rolling_statistics(data, ticker)

    # Seasonality and trend analysis
    seasonality_trend_analysis(data, ticker)

    # Risk analysis
    risk_analysis(data, ticker)

    # Show basic stats and insights summary
    print("\nBasic Statistics:\n", data.describe())
    print(f"\nSummary for {ticker}:")
    print("- Closing price trends show growth/volatility patterns over time.")
    print("- Daily returns and volatility patterns reflect risk level.")
    print("- Rolling mean and standard deviation provide trend insights.")
